In [1]:
import torch 
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, GPT2LMHeadModel
from torch.nn import CrossEntropyLoss

/Users/alexfeng/miniconda3/envs/esv1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ! pip install accelerate

In [6]:
class Ppl(object):
    def __init__(self,model_path) -> None:
        self.device = torch.device('cude:0' if torch.cuda.is_available() else 'cpu')
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_path, 
                                                          device_map="auto",
                                                        #   torch_dtype=torch.float16,
                                                          trust_remote_code=True)
        
        self.model.to(self.device)

    def cpmpute(self, text, max_length=50):
        inputs = self.tokenizer(text, padding="max_length",max_length=max_length, truncation=True, return_tensors="pt").to(self.device)
        # inputs = self.tokenizer(text, padding='max_length',max_length=max_length, truncation=True, return_tensors="pt").to(self.device)

        print("inputs:",inputs)
        bs, sl = inputs['input_ids'].size()
        print("bs:",bs)
        print("sl:",sl)
        # outputs = self.model(**inputs, labels = inputs['input_ids'])
        outputs = self.model(**inputs, labels = inputs['input_ids'])
        print("outputs:",outputs)
        
        logits=outputs[1]
        print("logits:",logits)

        shift_logits = logits[:, :-1, :].contiguous()
        print("shift_logits:",shift_logits)

        shift_labels = inputs["input_ids"][:,1:].contiguous()
        print("shitf_labels:",shift_labels)

        shift_attentions = inputs["attention_mask"][:, 1:].contiguous()
        print("shift_logits:",shift_attentions)

        loss_fct = CrossEntropyLoss(ignore_index=0,reduction="none")
        print("loss_fct:",loss_fct)

        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1)).detach().reshape(bs, -1)
        print("loss:",loss)

        meanloss=loss.sum(1) / shift_attentions.sum(1)
        print("meanloss:",meanloss)

        ppl = torch.exp(meanloss).cpu().numpy()
        return ppl

        
        

In [7]:
plm_path = "/Users/alexfeng/Desktop/alex/code/work_lr/deep_lr/hgmodel/gpt2_chinese_cluecorpussmall"
ppl_metric = Ppl(plm_path)

In [8]:
text = "你好"
ppl_metric.cpmpute(text=text)

inputs: {'input_ids': tensor([[ 101,  872, 1962,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])}
bs: 1
sl: 50
outputs: CausalLMOutputWithCrossAttentions(loss=tensor(13.5320, grad_fn=<NllLossBackward0>), logits=tensor([[[ -9.9143,  -9.7647,  -9.8217,  ...,  -9.6961,  -9.7799,  -9.6771],
         [-10.1990, -10.5101, -10.8450,  ...

array([153.19316], dtype=float32)